This file is to test my model on the TUH-EEG-seizure data

Here I predict X(t+1)-X(t) from GPT directly. (on Apr 13, 2023)

And I also change the network a bit.

1. add spatial attention

2. v = v + s(v)

In [1]:
run_python_script = False
model_name = "Lay2_dis_1step_latent_epoch1"

'Lay2_dis_1step_latent_epoch1'

In [2]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT, MODEL_ROOT

In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from easydict import EasyDict as edict
import time

if not run_python_script:
    plt.style.use(FIG_ROOT/"base.mplstyle")

In [4]:
import importlib
import data_utils.eeg_load
importlib.reload(data_utils.eeg_load)

<module 'data_utils.eeg_load' from '/data/rajlab1/user_data/jin/MyResearch/gTVDN-NN/notebooks/../mypkg/data_utils/eeg_load.py'>

In [5]:
import models.my_main_model_dis_latent
importlib.reload(models.my_main_model_dis_latent)

<module 'models.my_main_model_dis_latent' from '/data/rajlab1/user_data/jin/MyResearch/gTVDN-NN/notebooks/../mypkg/models/my_main_model_dis_latent.py'>

In [6]:
from models.my_main_model_dis_latent import myNet
from models.losses import ordinal_mse_loss, my_nllloss
from models.model_utils import generate_position_encode, load_model_class, snorm_cdf
from data_utils.eeg_load import EEG_data, MyDataLoader, digitize_data
from utils.misc import delta_time, load_pkl_folder2dict, save_pkl_dict2folder, truncated_mean_upper

In [7]:
# pkgs for pytorch (on Apr 3, 2023)
import torch
import torch.nn as nn
from torch.functional import F
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda")
else:
    torch.set_default_tensor_type(torch.DoubleTensor)
    device = torch.device("cpu")

# training

In [8]:
def trans_batch(batch, is_std=False):
    """transform the batch to make it easy for training
    """
    X, Y = batch[:, :-config.move_step], batch[:, config.move_step:]
    if is_std:
        Xstd = X.std(axis=(1, 2), keepdims=True)
        X = X/Xstd
    if is_std:
        return X, Y, Xstd
    else:
        return X, Y

In [9]:
paras = edict()
paras.nroi = 19

config = edict()
config.nfeature = paras.nroi # the dim of features at each time point
config.ndim = 256 # the output of the first FC layer
config.dropout = 0.5 # the dropout rate
config.n_layer = 2 # the number of self-attention layers
config.n_head = 8 # numher of heads for multi-head attention
config.is_mask = True # Use mask to make the attention causal
config.is_bias = True # Bias  for layernorm
config.block_size = 256 # the preset length of seq, 
config.batch_size = 1 # the batch size
config.move_step = 90 * 2 # from X_t to predict X_t+move_step
config.fs = 90
config.k = 8 # discretize to 2^k levels
config.target_dim = paras.nroi ** 2  # the target dim 

paras_train = edict()
paras_train.nepoch= 2
paras_train.loss_out = 10
paras_train.clip = 1 # 
paras_train.lr_step = 300
paras_train.test_loss_out = 50

In [10]:
train_data = EEG_data("train_train_health", "AR", 
                      move_dict=dict(winsize=config.block_size,
                                     stepsize=config.block_size,
                                     marginsize=None),
                      preprocess_dict=dict(is_detrend=True, 
                                      is_drop=True,
                                      target_fs=90, 
                                      filter_limit=[1, 45], 
                                      is_diff=False), 
                      scale_fct=None
                     )
test_data = EEG_data("train_test_health", "AR", 
                      move_dict=dict(winsize=config.block_size,
                                     stepsize=config.block_size,
                                     marginsize=None),
                      preprocess_dict=dict(is_detrend=True, 
                                      is_drop=True,
                                      target_fs=90, 
                                      filter_limit=[1, 45], 
                                      is_diff=False), 
                      scale_fct=None
                    )

train_data_loader = MyDataLoader(train_data, batch_size=config.batch_size, shuffle=True)
test_data_loader = MyDataLoader(test_data, batch_size=4, shuffle=False)
print(len(train_data_loader), len(test_data_loader))

545639 11490


In [11]:
data = train_data[10]
data_dis = digitize_data(data, config.k);
data = torch.tensor(data[None]).double();
data_dis = torch.tensor(data_dis[None]);

net = myNet(config);
output = net(data, data);

number of parameters: 1.69M


<All keys matched successfully>